# import package

In [1]:
import math
import h5py
import scipy
import random
import scipy.io as scio
from PIL import Image
from scipy import ndimage
from tensorflow.python.framework import ops

import time
import tensorflow as tf 
import numpy as np
import scipy.misc
import pydot
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import SVG

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import layer_utils
from keras.utils import plot_model
from keras.utils.data_utils import get_file
from keras.utils.vis_utils import model_to_dot

import keras.backend as K

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

%matplotlib inline
np.random.seed(1)



Using TensorFlow backend.


# 辅助函数

In [2]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def max_min_normalization(data_array):
    rows = data_array.shape[0]
    cols = data_array.shape[1]
    
    temp_array = np.zeros((rows,cols))
    col_min = data_array.min(axis=0)
    col_max = data_array.max(axis=0)

    for i in range(0,rows,1):
        for j in range(0,cols,1):
            temp_array[i][j] = (data_array[i][j]-col_min[j])/(col_max[j]-col_min[j])
    return temp_array


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((Y.shape[0],m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches



# load data

In [3]:
"下载数据和标签"
d = scio.loadmat('data1.mat')
data  = d['data']
label = d['label']
print('data shape = ',data.shape)
print('label shape = ',label.shape)

"随机打乱数据和标签"
N = data.shape[0]
index = np.random.permutation(N)
data  = data[index,:,:]
label = label[index,:]

"对数据data升维度,并且标签 one-hot"
data = np.expand_dims(data, axis=3)
label=label-1
label = convert_to_one_hot(label,52).T
print(data.shape, label.shape)

"选取训练样本、测试样本"
N = data.shape[0]
num_train = round(N*0.9)
num_test  = N-num_train

X_train = data[0:num_train,:,:,:]
Y_train = label[0:num_train,:]
X_test  = data[num_train:N,:,:,:]
Y_test  = label[num_train:N,:]

print(" ")
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))



data shape =  (11322, 16, 10)
label shape =  (11322, 1)
(11322, 16, 10, 1) (11322, 52)
 
number of training examples = 10190
number of test examples = 1132
X_train shape: (10190, 16, 10, 1)
Y_train shape: (10190, 52)
X_test shape: (1132, 16, 10, 1)
Y_test shape: (1132, 52)


# create model

In [4]:
def CNN_semg(input_shape=(16,10,1), classes=52):
    
    X_input = Input(input_shape)
    
    "block 1"
    "32 filters,  a row of the length of number of electrodes,  ReLU"
    X = Conv2D(filters=32, kernel_size=(1,10), strides=(1,1),padding='same', name='conv1')(X_input)
    X = Activation('relu', name='relu1')(X)
    
    "block 2"
    "32 filters 3*3,  ReLU,  average pool 3*3"
    X = Conv2D(filters=32, kernel_size=(3,3), strides=(1,1),padding='same', name='conv2')(X)
    X = Activation('relu', name='relu2')(X)
    X = AveragePooling2D((3,3), strides=(2,2), name='pool1')(X)
    
    "block 3"
    "64 filters 5*5,  ReLu,  average pool 3*3"
    X = Conv2D(filters=64, kernel_size=(5,5), strides=(1,1),padding='same', name='conv3')(X)
    X = Activation('relu', name='relu3')(X)
    X = AveragePooling2D((3,3), strides=(1,1), name='pool2')(X)
    
    "block 4"
    "64 filters 5*1,  ReLU"
    X = Conv2D(filters=64, kernel_size=(5,1), strides=(1,1),padding='same', name='conv4')(X)
    X = Activation('relu', name='relu4')(X)
    
    "block 5"
    "filters 1*1,  softmax loss"
    X = Conv2D(filters=32, kernel_size=(1,1), strides=(1,1),padding='same', name='conv5')(X)
    X = Flatten(name='flatten')(X)
    
    X = Dense(256,    activation='relu',    name='fc1')(X)
    X = Dense(classes, activation='softmax', name='fc2')(X)
    
    model = Model(inputs=X_input, outputs=X, name='CNN_semg')
    
    return model
    
    

# train & evaluate

In [5]:
model = CNN_semg(input_shape = (16, 10, 1), classes = 52)  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=64)


preds_train = model.evaluate(X_train, Y_train)
print("Train Loss = " + str(preds_train[0]))
print("Train Accuracy = " + str(preds_train[1]))

preds_test  = model.evaluate(X_test, Y_test)
print("Test Loss = " + str(preds_test[0]))
print("Test Accuracy = " + str(preds_test[1]))



Epoch 1/50
10190/10190 [==============================] - 13s 1ms/step - loss: 3.4699 - acc: 0.0977: 0s - loss: 3.4706 - acc: 0.097
Epoch 2/50
10190/10190 [==============================] - 14s 1ms/step - loss: 2.6670 - acc: 0.2596
Epoch 3/50
10190/10190 [==============================] - 12s 1ms/step - loss: 2.2667 - acc: 0.3616
Epoch 4/50
10190/10190 [==============================] - 11s 1ms/step - loss: 2.0363 - acc: 0.4272
Epoch 5/50
10190/10190 [==============================] - 11s 1ms/step - loss: 1.8261 - acc: 0.4815
Epoch 6/50
10190/10190 [==============================] - 11s 1ms/step - loss: 1.6929 - acc: 0.5152
Epoch 7/50
10190/10190 [==============================] - 11s 1ms/step - loss: 1.6029 - acc: 0.5312
Epoch 8/50
10190/10190 [==============================] - 13s 1ms/step - loss: 1.4795 - acc: 0.5588
Epoch 9/50
10190/10190 [==============================] - 13s 1ms/step - loss: 1.3895 - acc: 0.5877
Epoch 10/50
10190/10190 [==============================] - 13s 1ms/s

# plot model

In [6]:
"打印模型图层细节"
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 10, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 10, 32)        352       
_________________________________________________________________
relu1 (Activation)           (None, 16, 10, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 10, 32)        9248      
_________________________________________________________________
relu2 (Activation)           (None, 16, 10, 32)        0         
_________________________________________________________________
pool1 (AveragePooling2D)     (None, 7, 4, 32)          0         
_________________________________________________________________
conv3 (Conv2D)               (None, 7, 4, 64)          51264     
__________